<a href="https://colab.research.google.com/github/AnthonyJFeola/DS-Unit-2-Linear-Models/blob/master/module3-ridge-regression/LS_DS_213_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 1, Module 3*

---

# Ridge Regression

## Assignment

We're going back to our other **New York City** real estate dataset. Instead of predicting apartment rents, you'll predict property sales prices.

But not just for condos in Tribeca...

- [ ] Use a subset of the data where `BUILDING_CLASS_CATEGORY` == `'01 ONE FAMILY DWELLINGS'` and the sale price was more than 100 thousand and less than 2 million.
- [ ] Do train/test split. Use data from January — March 2019 to train. Use data from April 2019 to test.
- [ ] Do one-hot encoding of categorical features.
- [ ] Do feature selection with `SelectKBest`.
- [ ] Fit a ridge regression model with multiple features. Use the `normalize=True` parameter (or do [feature scaling](https://scikit-learn.org/stable/modules/preprocessing.html) beforehand — use the scaler's `fit_transform` method with the train set, and the scaler's `transform` method with the test set)
- [ ] Get mean absolute error for the test set.
- [ ] As always, commit your notebook to your fork of the GitHub repo.

The [NYC Department of Finance](https://www1.nyc.gov/site/finance/taxes/property-rolling-sales-data.page) has a glossary of property sales terms and NYC Building Class Code Descriptions. The data comes from the [NYC OpenData](https://data.cityofnewyork.us/browse?q=NYC%20calendar%20sales) portal.


## Stretch Goals

Don't worry, you aren't expected to do all these stretch goals! These are just ideas to consider and choose from.

- [ ] Add your own stretch goal(s) !
- [ ] Instead of `Ridge`, try `LinearRegression`. Depending on how many features you select, your errors will probably blow up! 💥
- [ ] Instead of `Ridge`, try [`RidgeCV`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html).
- [ ] Learn more about feature selection:
    - ["Permutation importance"](https://www.kaggle.com/dansbecker/permutation-importance)
    - [scikit-learn's User Guide for Feature Selection](https://scikit-learn.org/stable/modules/feature_selection.html)
    - [mlxtend](http://rasbt.github.io/mlxtend/) library
    - scikit-learn-contrib libraries: [boruta_py](https://github.com/scikit-learn-contrib/boruta_py) & [stability-selection](https://github.com/scikit-learn-contrib/stability-selection)
    - [_Feature Engineering and Selection_](http://www.feat.engineering/) by Kuhn & Johnson.
- [ ] Try [statsmodels](https://www.statsmodels.org/stable/index.html) if you’re interested in more inferential statistical approach to linear regression and feature selection, looking at p values and 95% confidence intervals for the coefficients.
- [ ] Read [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf), Chapters 1-3, for more math & theory, but in an accessible, readable way.
- [ ] Try [scikit-learn pipelines](https://scikit-learn.org/stable/modules/compose.html).

In [0]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [0]:
import pandas as pd
import pandas_profiling

# Read New York City property sales data
df = pd.read_csv(DATA_PATH+'condos/NYC_Citywide_Rolling_Calendar_Sales.csv')

# Change column names: replace spaces with underscores
df.columns = [col.replace(' ', '_') for col in df]

# SALE_PRICE was read as strings.
# Remove symbols, convert to integer
df['SALE_PRICE'] = (
    df['SALE_PRICE']
    .str.replace('$','')
    .str.replace('-','')
    .str.replace(',','')
    .astype(int)
)

In [0]:
# BOROUGH is a numeric column, but arguably should be a categorical feature,
# so convert it from a number to a string
df['BOROUGH'] = df['BOROUGH'].astype(str)

In [0]:
# Reduce cardinality for NEIGHBORHOOD feature

# Get a list of the top 10 neighborhoods
top10 = df['NEIGHBORHOOD'].value_counts()[:10].index

# At locations where the neighborhood is NOT in the top 10, 
# replace the neighborhood with 'OTHER'
df.loc[~df['NEIGHBORHOOD'].isin(top10), 'NEIGHBORHOOD'] = 'OTHER'

In [0]:
import warnings
warnings.filterwarnings('ignore')

In [6]:
df.head(5)

,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,EASE-MENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
0,1,OTHER,13 CONDOS - ELEVATOR APARTMENTS,2,716,1246,NaN,R4,"447 WEST 18TH STREET, PH12A",PH12A,10011.0,1.0,0.0,1.0,"10,733",1979.0,2007.0,2,R4,0,01/01/2019
1,1,OTHER,21 OFFICE BUILDINGS,4,812,68,NaN,O5,144 WEST 37TH STREET,NaN,10018.0,0.0,6.0,6.0,"2,962",15435.0,1920.0,4,O5,0,01/01/2019
2,1,OTHER,21 OFFICE BUILDINGS,4,839,69,NaN,O5,40 WEST 38TH STREET,NaN,10018.0,0.0,7.0,7.0,"2,074",11332.0,1930.0,4,O5,0,01/01/2019
3,1,OTHER,13 CONDOS - ELEVATOR APARTMENTS,2,592,1041,NaN,R4,"1 SHERIDAN SQUARE, 8C",8C,10014.0,1.0,0.0,1.0,0,500.0,0.0,2,R4,0,01/01/2019
4,1,UPPER EAST SIDE (59-79),15 CONDOS - 2-10 UNIT RESIDENTIAL,2C,1379,1402,NaN,R1,"20 EAST 65TH STREET, B",B,10065.0,1.0,0.0,1.0,0,6406.0,0.0,2,R1,0,01/01/2019


In [0]:
df = df[(df['BUILDING_CLASS_CATEGORY'] == '01 ONE FAMILY DWELLINGS') & (df['SALE_PRICE'] > 100000) & (df['SALE_PRICE'] < 2000000)]

In [0]:
df['SALE_DATE'] = pd.to_datetime(df['SALE_DATE'], infer_datetime_format=True)
df['LAND_SQUARE_FEET'] = (df['LAND_SQUARE_FEET'] !='n').astype(int)

In [0]:
df['SALE_YEAR'] = df['SALE_DATE'].dt.year
df['SALE_MONTH'] = df['SALE_DATE'].dt.month

In [0]:
train = df[(df['SALE_MONTH'] == 1) | (df['SALE_MONTH'] == 2) | (df['SALE_MONTH'] == 3)]
test = df[df['SALE_MONTH'] == 4]
train.reset_index(drop=True, inplace=True)
test.reset_index(drop=True, inplace=True)

In [0]:
from pandas_profiling import ProfileReport

In [0]:
train_profile = ProfileReport(train)
test_profile = ProfileReport(test)

In [13]:
train_profile

Number of variables,23
Number of observations,2507
Total Missing (%),8.7%
Total size in memory,450.6 KiB
Average record size in memory,184.1 B
Numeric,9
Categorical,7
Boolean,1
Date,1
Text (Unique),0
Rejected,5


In [14]:
test_profile

Number of variables,23
Number of observations,644
Total Missing (%),8.7%
Total size in memory,115.8 KiB
Average record size in memory,184.2 B
Numeric,8
Categorical,6
Boolean,1
Date,1
Text (Unique),0
Rejected,7


In [15]:
train.describe()

,BLOCK,LOT,EASE-MENT,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_YEAR,SALE_MONTH
count,2507.000000,2507.000000,0.0,2507.000000,2507.000000,2507.000000,2507.000000,2507.0,2507.000000,2507.000000,2507.0,2.507000e+03,2507.0,2507.000000
mean,6758.303949,75.778221,NaN,10993.398484,0.987635,0.016354,1.003989,1.0,1473.744715,1944.766653,1.0,6.215737e+05,2019.0,1.940965
std,3975.909029,157.531138,NaN,494.291462,0.110532,0.129966,0.171794,0.0,599.217635,27.059337,0.0,2.916072e+05,0.0,0.832611
min,21.000000,1.000000,NaN,10301.000000,0.000000,0.000000,0.000000,1.0,0.000000,1890.000000,1.0,1.040000e+05,2019.0,1.000000
25%,3837.500000,21.000000,NaN,10314.000000,1.000000,0.000000,1.000000,1.0,1144.000000,1925.000000,1.0,4.405000e+05,2019.0,1.000000
50%,6022.000000,42.000000,NaN,11234.000000,1.000000,0.000000,1.000000,1.0,1368.000000,1940.000000,1.0,5.600000e+05,2019.0,2.000000
75%,9888.500000,70.000000,NaN,11413.000000,1.000000,0.000000,1.000000,1.0,1683.000000,1960.000000,1.0,7.500000e+05,2019.0,3.000000
max,16350.000000,2720.000000,NaN,11697.000000,1.000000,2.000000,3.000000,1.0,7875.000000,2018.000000,1.0,1.955000e+06,2019.0,3.000000


In [16]:
train.describe(exclude='number')

,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_DATE
count,2507,2507,2507,2507,2507,2507,1,2507,2507
unique,5,6,1,2,13,2497,1,11,68
top,4,OTHER,01 ONE FAMILY DWELLINGS,1,A1,294 FREEBORN STREET,RP.,A1,2019-01-31 00:00:00
freq,1204,2382,2507,2476,919,2,1,919,78
first,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-01-01 00:00:00
last,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-03-30 00:00:00


In [17]:
target = 'SALE_PRICE'
high_cardinality_or_drop = ['BUILDING_CLASS_CATEGORY', 'TAX_CLASS_AT_PRESENT', 
                            'BUILDING_CLASS_AT_PRESENT', 'ADDRESS', 
                            'APARTMENT_NUMBER', 'EASE-MENT', 'COMMERCIAL_UNITS', 
                            'TAX_CLASS_AT_TIME_OF_SALE', 'SALE_YEAR', 'SALE_MONTH', 
                            'SALE_DATE', 'BUILDING_CLASS_AT_TIME_OF_SALE' ]
features = train.columns.drop([target] + high_cardinality_or_drop)
features

Index(['BOROUGH', 'NEIGHBORHOOD', 'BLOCK', 'LOT', 'ZIP_CODE',
       'RESIDENTIAL_UNITS', 'TOTAL_UNITS', 'LAND_SQUARE_FEET',
       'GROSS_SQUARE_FEET', 'YEAR_BUILT'],
      dtype='object')

In [0]:
X_train = train[features]
y_train = train[target]
X_test = test[features]
y_test = test[target]

In [19]:
X_train.head()

,BOROUGH,NEIGHBORHOOD,BLOCK,LOT,ZIP_CODE,RESIDENTIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT
0,3,OTHER,5495,801,11230.0,1.0,1.0,1,1325.0,1930.0
1,4,OTHER,7918,72,11427.0,1.0,1.0,1,2001.0,1940.0
2,2,OTHER,4210,19,10461.0,1.0,1.0,1,2043.0,1925.0
3,3,OTHER,5212,69,11226.0,1.0,1.0,1,2680.0,1899.0
4,3,OTHER,7930,121,11203.0,1.0,1.0,1,1872.0,1940.0


In [0]:
import category_encoders as ce 

encoder = ce.one_hot.OneHotEncoder(use_cat_names=True)
X_train_enc = encoder.fit_transform(X_train)
X_test_enc = encoder.transform(X_test)

In [21]:
X_test_enc

,BOROUGH_3,BOROUGH_4,BOROUGH_2,BOROUGH_5,BOROUGH_1,NEIGHBORHOOD_OTHER,NEIGHBORHOOD_FLUSHING-NORTH,NEIGHBORHOOD_BEDFORD STUYVESANT,NEIGHBORHOOD_FOREST HILLS,NEIGHBORHOOD_BOROUGH PARK,NEIGHBORHOOD_ASTORIA,BLOCK,LOT,ZIP_CODE,RESIDENTIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT
0,0,0,1,0,0,1,0,0,0,0,0,5913,878,10471.0,1.0,1.0,1,2272.0,1930.0
1,0,0,1,0,0,1,0,0,0,0,0,5488,48,10465.0,1.0,1.0,1,720.0,1935.0
2,1,0,0,0,0,1,0,0,0,0,0,5936,31,11209.0,1.0,1.0,1,2210.0,1925.0
3,1,0,0,0,0,1,0,0,0,0,0,7813,24,11210.0,1.0,1.0,1,1520.0,1915.0
4,1,0,0,0,0,1,0,0,0,0,0,8831,160,11229.0,1.0,1.0,1,840.0,1925.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
639,0,1,0,0,0,1,0,0,0,0,0,13215,3,11422.0,1.0,1.0,1,1478.0,1925.0
640,0,1,0,0,0,1,0,0,0,0,0,11612,73,11420.0,1.0,1.0,1,1280.0,1930.0
641,0,1,0,0,0,1,0,0,0,0,0,11808,50,11420.0,1.0,1.0,1,1333.0,1945.0
642,0,1,0,0,0,1,0,0,0,0,0,12295,23,11434.0,1.0,1.0,1,1020.0,1935.0


In [0]:
from sklearn.feature_selection import SelectKBest, f_regression

selector = SelectKBest(score_func=f_regression, k=10)
X_train_kbest = selector.fit_transform(X_train_enc, y_train)
X_test_kbest = selector.transform(X_test_enc)

In [23]:
X_train_kbest.shape, X_test_kbest.shape

((2507, 10), (644, 10))

In [24]:
X_train_kbest

array([[1.0000e+00, 0.0000e+00, 0.0000e+00, ..., 1.1230e+04, 1.0000e+00,
        1.3250e+03],
       [0.0000e+00, 0.0000e+00, 0.0000e+00, ..., 1.1427e+04, 1.0000e+00,
        2.0010e+03],
       [0.0000e+00, 1.0000e+00, 0.0000e+00, ..., 1.0461e+04, 1.0000e+00,
        2.0430e+03],
       ...,
       [0.0000e+00, 0.0000e+00, 1.0000e+00, ..., 1.0302e+04, 1.0000e+00,
        1.8070e+03],
       [0.0000e+00, 0.0000e+00, 1.0000e+00, ..., 1.0305e+04, 1.0000e+00,
        6.2100e+02],
       [0.0000e+00, 0.0000e+00, 0.0000e+00, ..., 1.1429e+04, 1.0000e+00,
        1.1630e+03]])

In [25]:
mask = selector.get_support()
mask

array([ True, False,  True,  True, False,  True,  True, False,  True,
       False, False,  True, False,  True, False,  True, False,  True,
       False])

In [26]:
X_train_enc.columns[mask]

Index(['BOROUGH_3', 'BOROUGH_2', 'BOROUGH_5', 'NEIGHBORHOOD_OTHER',
       'NEIGHBORHOOD_FLUSHING-NORTH', 'NEIGHBORHOOD_FOREST HILLS', 'BLOCK',
       'ZIP_CODE', 'TOTAL_UNITS', 'GROSS_SQUARE_FEET'],
      dtype='object')

In [27]:
X_train_enc.columns[~mask]

Index(['BOROUGH_4', 'BOROUGH_1', 'NEIGHBORHOOD_BEDFORD STUYVESANT',
       'NEIGHBORHOOD_BOROUGH PARK', 'NEIGHBORHOOD_ASTORIA', 'LOT',
       'RESIDENTIAL_UNITS', 'LAND_SQUARE_FEET', 'YEAR_BUILT'],
      dtype='object')

In [28]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

mae_list = []
for k in range(1, X_train_enc.shape[1]+1):
  print(f'{k} features')
  selector = SelectKBest(score_func=f_regression, k=k)
  X_train_kbest = selector.fit_transform(X_train_enc, y_train)
  X_test_kbest = selector.transform(X_test_enc)
  model = LinearRegression()
  model.fit(X_train_kbest, y_train)
  y_pred = model.predict(X_test_kbest)
  mae = mean_absolute_error(y_pred, y_test)
  print(f'MAE on test set: ${mae:.2f}')
  mae_list.append(mae)

1 features
MAE on test set: $183640.59
2 features
MAE on test set: $174495.92
3 features
MAE on test set: $175142.68
4 features
MAE on test set: $173620.37
5 features
MAE on test set: $174228.72
6 features
MAE on test set: $174011.33
7 features
MAE on test set: $169628.18
8 features
MAE on test set: $170425.59
9 features
MAE on test set: $169744.84
10 features
MAE on test set: $162186.98
11 features
MAE on test set: $160380.14
12 features
MAE on test set: $160405.90
13 features
MAE on test set: $161706.15
14 features
MAE on test set: $162274.12
15 features
MAE on test set: $162294.29
16 features
MAE on test set: $162310.21
17 features
MAE on test set: $162310.21
18 features
MAE on test set: $162310.21
19 features
MAE on test set: $162310.21


In [29]:
import matplotlib.pyplot as plt
from sklearn.linear_model import Ridge
from IPython.display import display, HTML
for alpha in [0.001, 0.01, 0.1, 1.0, 1, 100.0, 1000.0]:
    
    # Fit Ridge Regression model
    display(HTML(f'Ridge Regression, with alpha={alpha}'))
    model = Ridge(alpha=alpha, normalize=True)
    model.fit(X_train_enc, y_train)
    y_pred = model.predict(X_test_enc)

    # Get Test MAE
    mae = mean_absolute_error(y_test, y_pred)
    display(HTML(f'Test Mean Absolute Error: ${mae:,.0f}'))
    
    # Plot coefficients
    coefficients = pd.Series(model.coef_, X_train_enc.columns)
    plt.figure(figsize=(16,8))
    coefficients.sort_values().plot.barh(color='grey')
    plt.xlim(-10000,10000)
    plt.show()